# This notebook is used to run the python code through Jupyter as if it was a command line. So we run python files, and can use commands such as mpirun to run stuff in parallel.
# Need to put a '!' in fron of every command meant for the command line

In [ ]:
#Converting the notebook to a runnable python file 
!jupyter nbconvert --to script MONDtest_NoFunctions_MPI.ipynb

In [ ]:
#Running the full script in parallel using MPI. Optimal number of processes:6
! mpirun -n 6 python3 MONDtest_NoFunctions_MPI.py